In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import QFT, UnitaryGate

def create_qpe_circuit(num_ancilla: int,
                       U_gate: UnitaryGate,
                       state_prep: QuantumCircuit) -> QuantumCircuit:

    n_target = state_prep.num_qubits
    U_mat    = U_gate.to_matrix()

    qc = QuantumCircuit(num_ancilla + n_target, num_ancilla)

    qc.h(range(num_ancilla))
    qc.compose(state_prep,
               qubits=list(range(num_ancilla,
                                 num_ancilla + n_target)),
               inplace=True)

    for j in range(num_ancilla):
        mat_pow = np.linalg.matrix_power(U_mat, 2**j)
        cU = UnitaryGate(mat_pow).control()
        qc.append(cU,
                  [j] + list(range(num_ancilla,
                                   num_ancilla + n_target)))

    qc.compose(QFT(num_ancilla, do_swaps=True).inverse(),
               range(num_ancilla), inplace=True)

    qc.measure(range(num_ancilla), range(num_ancilla))
    return qc

import numpy as np
from qiskit import transpile
from qiskit_aer import Aer
from qiskit.circuit.library import UnitaryGate

state2 = QuantumCircuit(2)
state2.h([0,1])                  
U2 = UnitaryGate(np.eye(4), label="I2")
qc2 = create_qpe_circuit(3, U2, state2)
print("2-qubit total qubits:", qc2.num_qubits)  

state4 = QuantumCircuit(4)
state4.h(range(4))                  
U4 = UnitaryGate(np.eye(16), label="I4")
qc4 = create_qpe_circuit(3, U4, state4)
print("4-qubit total qubits:", qc4.num_qubits)


2-qubit total qubits: 5
4-qubit total qubits: 7


C:\Users\82108\AppData\Local\Temp\ipykernel_31180\2180072433.py:41: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  qc.compose(QFT(num_ancilla, do_swaps=True).inverse(),
